### Load packages etc, and load the child-class of gmspython, "Abatement", and load "techdata_to_tree" which converts a technology catalog into production trees

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import abatement, sys, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data'
gams_folder = data_folder + "\\..\\gamsmodels\\Main"
# pickle_path = 

In [2]:
m = abatement.abate(pickle_path=gams_folder+"/gmspython_Abatement", work_folder=work_folder)

In [3]:
m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001

m.write_and_run()
if m.model_instances["baseline"].modelstat == 16.0:
    print("\nSuccess! The modelstat was 16.0")


Success! The modelstat was 16.0


#### Sneaky solve to adjust sigmas into desired values

The model could be solved using elasticities of 0.0001. Let's check if it can be solved when these are set to higher numbers, implying a solution farther from the starting values (which, as mentioned, were set based on Leontief).

In [4]:
# condition = m.model.settings.databases["Abatement"]["sigma"].vals.index.str.startswith("C_E") & ~m.model.settings.databases["Abatement"]["sigma"].vals.index.str.contains("base")

In [5]:
# for sigma_val in range(0, 60, 10):
#     sigma_val = sigma_val/100
#     if sigma_val == 0:
#         sigma_val = 0.0001
# #    for MNL_sigma in [0.1, 1]:

#     #Set values
#     #m.model.settings.databases["Abatement"]["sigma"].vals.loc[condition] = MNL_sigma
#     #m.model.settings.databases["Abatement"]["sigma"].vals.loc[~condition] = sigma_val
#     m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = sigma_val
#     m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -sigma_val

#     #Try to solve
#     m.write_and_run()

#     #How did it go?
#     if m.model_instances["baseline"].modelstat == 16.0:
#         print("\nSUCCESS:")
#         print("Sigma/eta:", round(sigma_val, 4))
#         #print("Sigma/eta (MNL):", round(MNL_val, 4))
#     else:
#         print("\nFAIL:")
#         print("Sigma/eta:", round(sigma_val, 4))
#         #print("Sigma/eta: (MNL)", round(MNL_sigma, 4))

As can be seen, the model cannot be solved when the elasticities get too large. To accomodate this, we replace each iteration's starting values with the previous iteration's solution, while gradually increasing $\sigma$ and $\mu$.
This allows us to set these to (hopefully) any value. This "gradual changing of parameters" is called 'sneaky solve'. :)

### Sneaky solve: Gradual increase in sigma

In [6]:
db_star = DataBase.GPM_database()

In [7]:
sigma_star = m.get("sigma").copy()
eta_star = m.get("eta").copy()

In [8]:
sigma_star

n
U_ID_C_EL_3_base    0.0001
ER                  0.0001
ID_2                0.0001
C_EH_3              0.0001
IO_tech             0.0001
EL                  0.0001
ID_4                0.0001
ID_5                0.0001
U_ID_C_EL_2_base    0.0001
C_SO2_1             0.0001
C_EH_1              0.0001
EOP_t1              0.0001
U_ID_C_EL_4_base    0.0001
C_EL_3              0.0001
EH                  0.0001
C_EH_2              0.0001
C_ER_2              0.0001
C_EL_2              0.0001
C_EL_4              0.0001
U_ID_C_EL_1_base    0.0001
C_EL_5              0.0001
C_EL_1              0.0001
U_ID_C_EH_1_base    0.0001
C_ER_1              0.0001
ID_1                0.0001
ID_3                0.0001
Name: sigma, dtype: float64

In [9]:
#Energy services combine their underlying components with mild substitution possibilities
sigma_star[m.get("kno_ID_EC")] = 0.5

In [10]:
#All CET splits get more substitution
eta_star[:] = -1

In [11]:
#The IO technology treats its outputs as perfect substitutes. This ensures they have the same price in equiliibrum.
eta_star["IO_tech"] = -10

In [12]:
#Components have a large degree of substitution between overlapping technology goods:
sigma_star[m.get("kno_ID_CU")] = 5

In [13]:
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star

Run again, while adding a checkpoint that we can use afterwards

In [18]:
m.checkpoints

{'leontief': <gams.execution.GamsCheckpoint at 0x279e4bf4f48>}

In [17]:
m.model_instances

{'baseline': <DB2Gams_l3.gams_model at 0x279e4e08248>}

In [19]:
name = "sneaky"
#Fejler med noget "delimiter" fejl
#m.write_and_run(name="test", add_checkpoint="test", options_add={"checkpoint":m.checkpoints["leontief"]})
m.write_and_run(name=name, add_checkpoint="leontief")

In [20]:
m.checkpoints[name] = m.model_instances[name].ws.add_checkpoint()

In [21]:
m.checkpoints

{'leontief': <gams.execution.GamsCheckpoint at 0x279e4e37508>,
 'sneaky': <gams.execution.GamsCheckpoint at 0x279e4eace48>}

Solve sneakily, gradually increasing sigmas towards their desired values (which themselves are stated in `db_star`)

In [22]:
#m.model_instances[name].solve_sneakily(db_star=db_star, options_run={"checkpoint":name})
#DETTE OPDATERER IKKE CHECKPOINTET "calibrated". VIRKER OGSÅ SPØJST AT MAN SKAL GIVE ET CHECKPOINT NÅR NU IDEEN ER AT DEN SKAL OPRETTE ET BASERET PÅ HVOR DEN ENDER.
# OG JEG HAR JO ALLEREDE GIVET DEN INIT-tingene. 

#gm.checkpoints[name] = mi.ws.add_checkpoint()
m.model_instances[name].solve_sneakily(db_star=db_star, from_cp=True, cp_init=m.checkpoints["leontief"], options_run={"checkpoint":m.checkpoints[name]})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [139]:
mi = m.model_instances[name]
# newjob = mi.ws.add_job_from_string("sets a,b;",**{'checkpoint': m.checkpoints[name]})
# newjob.run()

Check if the model ran succesfully:

In [26]:
if m.model_instances[name].out_db.series["Abatement_B_modelstat"].vals == 16:
    print("Ran successfully with modelstat " + str(m.model_instances[name].out_db.series["Abatement_B_modelstat"].vals))

Ran successfully with modelstat 16.0


The changes in e.g. demanded quantities, from the initial point (the solution where sigmas were ~0) are:

In [27]:
m.model_instances[name].out_db["qD"].vals - m.get("qD")

n
C_EH_1                   1.018307
C_EH_2                  -0.086536
C_EH_3                  -0.168528
C_EH_base               -0.763243
C_EL_1                   0.279783
                           ...   
U_ID_C_EL_4_base         2.729239
U_ID_C_EL_4_base_inp4    2.729239
U_ID_C_EL_5_base         0.882456
U_ID_C_ER_1_base         0.945925
U_ID_C_ER_2_base         0.949993
Name: qD, Length: 77, dtype: float64

while demand itself ended in the following quantities:

In [29]:
m.model_instances[name].out_db["qD"].vals

n
ID_5_oil                 1.078243e-08
U_ID_2_2                 1.023244e-11
U_ID_5_1                 1.804221e-08
ID_2                     8.090549e-11
IO_tech_electricity      4.249114e+01
                             ...     
ID_5                     2.695173e-08
C_EH_2                   2.913464e+00
ID_4_inp5                3.436840e-11
U_ID_C_EH_1_base_inp3    1.601831e+01
U_ID_C_EL_1_base         5.279783e+00
Name: qD, Length: 77, dtype: float64

And let's confirm that our sigmas actually ended in the desired spots:

In [30]:
m.model_instances[name].out_db["sigma"].vals

n
ID_5_oil                 0.0000
U_ID_2_2                 0.0000
ER                       0.5000
U_ID_5_1                 0.0000
ID_2                     0.0001
                          ...  
ID_5                     0.0001
C_EH_2                   5.0000
ID_4_inp5                0.0000
U_ID_C_EH_1_base_inp3    0.0000
U_ID_C_EL_1_base         0.0001
Name: sigma, Length: 96, dtype: float64

In [32]:
db = m.model_instances['sneaky'].out_db
m.model.database.merge_dbs(m.model.database,db,'second')
m.reset_settings()

In [33]:
m.setstate("calibrate")

In [34]:
# m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001
#m.model.settings.databases["Abatement"]["sigma"].vals[condition] = 0.2
#m.write_and_run(options_run={'output':sys.stdout})
m.write_and_run(overwrite=True)
if m.model_instances["baseline"].modelstat == 2.0:
    print("\nNPL problem solved successfully! The modelstat was 2.0")


NPL problem solved successfully! The modelstat was 2.0


## A shock to the oil price
We present graphically what happens when the oil price gradually increases (static model solution for each price).
For this, we again use the sneaky solve, but we make sure to extract variables of interest for each iteration, in contrast to above where we were not interested in the solutions per se, but simply that the values of $\sigma$ ended in the desired spots.

In [121]:
#mideleritid start fra starten
#m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.1
#m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.1

In [122]:
# m.write_and_run(name="calibrated", add_checkpoint="calibrated")

In [123]:
m.model_instances["calibrated"].out_db.get("sigma")

n
C_CO2_1             0.50
C_EH_1              1.20
C_EH_2              1.20
C_EH_3              1.20
C_EH_base           0.01
C_EL_1              1.20
C_EL_2              1.20
C_EL_3              1.20
C_EL_4              1.20
C_EL_5              1.20
C_EL_base           0.01
C_ER_1              1.20
C_ER_2              1.20
C_ER_base           0.01
EH                  0.01
EL                  0.01
EOP_tau1            0.50
ER                  0.01
ID_1                0.01
ID_2                0.01
ID_3                0.01
ID_4                0.01
ID_5                0.01
U_ID_C_EH_1_base    0.01
U_ID_C_EH_2_base    0.01
U_ID_C_EH_3_base    0.01
U_ID_C_EL_1_base    0.01
U_ID_C_EL_2_base    0.01
U_ID_C_EL_3_base    0.01
U_ID_C_EL_4_base    0.01
U_ID_C_EL_5_base    0.01
U_ID_C_ER_1_base    0.01
U_ID_C_ER_2_base    0.01
Name: sigma, dtype: float64

In [124]:
out_db = m.model_instances["calibrated"].out_db

In [125]:
m.write_and_run(name="oilshock")
#db_out = m.model_instances["baseline"].out_db

In [126]:
#m.write_and_run(name='taxshock', add_checkpoint='taxshock', overwrite=True)
mi = m.model_instances['oilshock']

In [127]:
#mi.out_db.series.__dict__

*2: Define structure of shock* 

In [128]:
db_end_of_loop = DataBase.GPM_database()

In [129]:
cond = out_db.get("PwT").index.str.contains("oil")
db_end_of_loop["PwT"] = out_db.get("PwT")[cond] + 12
#end_prices[cond] = out_db.get("PwT")[cond] + 5
#end_prices = end_prices[cond]

In [130]:
db_end_of_loop.series["PwT"].vals

n
C_EH_base_oil           13.0
C_EL_base_oil           13.0
C_ER_base_oil           13.0
ID_1_oil                13.0
ID_2_oil                13.0
ID_5_oil                13.0
U_ID_C_EH_1_base_oil    13.0
U_ID_C_EH_2_base_oil    13.0
U_ID_C_EH_3_base_oil    13.0
U_ID_C_EL_1_base_oil    13.0
U_ID_C_EL_2_base_oil    13.0
U_ID_C_EL_3_base_oil    13.0
U_ID_C_EL_4_base_oil    13.0
U_ID_C_EL_5_base_oil    13.0
U_ID_C_ER_1_base_oil    13.0
U_ID_C_ER_2_base_oil    13.0
Name: PwT, dtype: float64

In [131]:
#NY ME DDIREKTE
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["calibrated"])

In [132]:
(shock_db, shock_kwargs) = ShockFunction.sneaky_db(out_db, db_end_of_loop, n_steps = 25, loop_name='oil_loop')

*3: Define the solution we wish to store for each iteration (conditions argument is optional, but here we extract only the production sector goods):*

In [133]:
shock_db.series["PwT_oil_loop_subset"].vals

Index(['C_EH_base_oil', 'C_EL_base_oil', 'C_ER_base_oil', 'ID_1_oil',
       'ID_2_oil', 'ID_5_oil', 'U_ID_C_EH_1_base_oil', 'U_ID_C_EH_2_base_oil',
       'U_ID_C_EH_3_base_oil', 'U_ID_C_EL_1_base_oil', 'U_ID_C_EL_2_base_oil',
       'U_ID_C_EL_3_base_oil', 'U_ID_C_EL_4_base_oil', 'U_ID_C_EL_5_base_oil',
       'U_ID_C_ER_1_base_oil', 'U_ID_C_ER_2_base_oil'],
      dtype='object', name='n')

In [134]:
store_sol = {'qD': {'domains': shock_kwargs['loop_name']}}

*4: Create Shock instance*

In [135]:
shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'], store_sol=store_sol)
#shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'])
#solve_sneakily skal have selve checkpointet
#Gør noget så sneaky solve laver et sidste checkpoint

*5: Execute shock from the 'calibrated' checkpoint i.e. the model state after the sigmas have been chosen:*

In [136]:
mi.execute_shock_from_cp(shock=shock, cp=m.checkpoints['calibrated'])

In [137]:
sol_qD = mi.out_db.series["sol_qD"].vals

In [138]:
#tech_names = ["ID_thighest", "ID_thigher", "ID_tmedium", "ID_tlower", "ID_tlowest"]
#sol_qD.loc(axis=0)["oil_loop_1", tech_names]

In [139]:
sol_qD.reset_index().to_excel(os.getcwd() + "\\oil_price_shock_" + inputfile.split("_")[-1], index=False)

In [140]:
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["oilshock"])

The solution is stored as a parameter noted as "sol_"+ the name of the variable. In our case, the specific quantities are stored:

### **3.1 Diagnostics (dated)**
When the model cannot be solved, we run some diagnostics:

In [ ]:
# for symbol in ["PbT", "PwT", "qD"]:
#     start_vals = m.model_instances["baseline"].out_db.series["load_" + symbol].vals
#     end_vals = m.model_instances["baseline"].out_db.series[symbol].vals
#     rel_changes = (end_vals - start_vals)/start_vals
#     large_changes = rel_changes[~(rel_changes.between(-0.0001, 0.0001))]
#     if len(large_changes) > 0:
#         print("Large deviations for symbol \n" + symbol + ":")
#         print(large_changes, "\n")
#     else:
#         print("No large deviations for symbol: " + symbol + ".\n")

In [ ]:
# from decimal import Decimal

In [ ]:
# #Demands
# print(Decimal(qD["U_EOP_tau1_1"] - mu[("U_EOP_tau1_1", "C_CO2_1")] * qS["C_CO2_1"]))
# print(Decimal(qD["EOP_tau1"] - mu.loc[("U_EOP_tau1_1", "EOP_tau1")] * qD["U_EOP_tau1_1"]))
# print(Decimal(qD["EOP_tau1_electricity"] - mu[("EOP_tau1_electricity", "EOP_tau1")] * qD["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1_K"] - mu[("EOP_tau1_K", "EOP_tau1")] * qD["EOP_tau1"]))
# #zero profits
# print(Decimal(qS["C_CO2_1"]*PbT["C_CO2_1"] - qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"]))
# print(Decimal(qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"] - qD["EOP_tau1"]*PwT["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1"]*PwT["EOP_tau1"] - qD["EOP_tau1_electricity"]*PwT["EOP_tau1_electricity"] - qD["EOP_tau1_K"]*PwT["EOP_tau1_K"]))